# 构建你的循环神经网络（RNN）：逐步实现

本节，你将使用 **numpy** 实现你的第一个循环神经网络（RNN）。

循环神经网络（RNN）在自然语言处理（NLP）和其他序列任务中非常有效，因为它们具有“记忆”能力。  
RNN 可以逐个读取输入 $x^{\langle t \rangle}$（例如单词），并通过隐藏层的激活值在不同时间步之间传递信息或上下文。  
这使得单向 RNN 能够利用**过去的信息**来处理后续输入；而双向 RNN 则可以同时利用**过去与未来**的上下文信息。

---

**符号说明**：

- 上标 $[l]$ 表示与第 $l$ 层相关的对象。  
  - 例如：$a^{[4]}$ 表示第 4 层的激活值，$W^{[5]}$ 和 $b^{[5]}$ 表示第 5 层的参数。

- 上标 $(i)$ 表示与第 $i$ 个样本相关的对象。  
  - 例如：$x^{(i)}$ 表示第 $i$ 个训练样本的输入。

- 上标 $\langle t \rangle$ 表示与第 $t$ 个时间步相关的对象。  
  - 例如：$x^{\langle t \rangle}$ 表示时间步 $t$ 的输入 $x$，  
    而 $x^{(i)\langle t \rangle}$ 表示第 $i$ 个样本在时间步 $t$ 的输入。

- 下标 $i$ 表示向量的第 $i$ 个分量。  
  - 例如：$a^{[l]}_i$ 表示第 $l$ 层激活值向量中的第 $i$ 个分量。

---

我们假设你已经熟悉 `numpy`，并且已完成之前课程的学习。  
现在，让我们开始吧！


首先，让我们导入在本次作业中所需的所有软件包。

In [80]:
# 导入 numpy 库，并使用别名 np
# numpy 是 Python 中非常重要的科学计算库，提供了高效的数组操作、矩阵运算以及大量数学函数
# 这里使用别名 np 是为了简化后续代码书写，例如 np.array() 代替 numpy.array()
import numpy as np

# 从 rnn_utils 模块中导入所有函数和类
# rnn_utils 是自定义或第三方的工具模块，通常用于封装与循环神经网络（RNN）相关的函数
# 例如可能包含：初始化参数、前向传播、反向传播、梯度检查等函数
# 使用 '*' 表示导入模块中所有公开的函数、类和变量
#from rnn_utils import *

### 前置函数

In [81]:
"""
从零实现的简单字符级 RNN（NumPy 实现）
包含：
- softmax / 平滑损失 / 打印采样
- 参数初始化
- RNN 单步前向和反向（rnn_step_forward / rnn_step_backward）
- 整序列前向与反向（rnn_forward / rnn_backward）
- 参数更新
可直接复制到 Jupyter Notebook 作为教学模块使用。
"""

def softmax(x):
    """
    计算 softmax 概率分布（数值稳定版）
    
    参数：
        x -- numpy 数组，形状 (vocab_size, m) 或 (vocab_size, 1)
    返回：
        softmax 输出，形状与 x 相同，每列是一个概率分布
    说明：
        为了数值稳定，减去列最大值 np.max(x)。
        这里 sum 的 axis=0 表示对每一列（每个样本）做 softmax。
    """
    # 为了数值稳定性：减去每列的最大值，避免 exp 的溢出
    e_x = np.exp(x - np.max(x))
    # 每列归一化，使每列和为 1
    return e_x / e_x.sum(axis=0)


def smooth(loss, cur_loss):
    """
    对损失值做指数平滑（moving average），用于跟踪训练过程中的损失变化使其更平滑。
    
    参数：
        loss -- 之前保存的平滑损失（标量）
        cur_loss -- 当前批次/序列计算得到的实际损失（标量）
    返回：
        更新后的平滑损失（标量）
    
    解释：
        采用系数 0.999/0.001 做加权平均，长期保留历史信息，但对新损失略微响应。
    """
    return loss * 0.999 + cur_loss * 0.001


def print_sample(sample_ix, ix_to_char):
    """
    根据索引序列打印生成的字符序列（用于展示采样结果）。
    
    参数：
        sample_ix -- 索引列表，如 [4, 15, 2, ...]
        ix_to_char -- 索引到字符的字典映射（例如 {0:'a', 1:'b', ...}）
    """
    # 将每个索引映射为字符并拼接为字符串
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    # 格式化打印，便于在 Notebook 中观察
    print('----\n %s \n----' % (txt, ))


def get_initial_loss(vocab_size, seq_length):
    """
    计算初始损失值的估计（用于平滑初始损失的基线）。
    
    参数：
        vocab_size -- 词汇表大小（整数）
        seq_length -- 期望的序列长度（例如每次采样的字符数或样本数）
    返回：
        初始损失（标量），等于 -log(1/vocab_size) * seq_length，
        表示如果每一步都随机且均匀采样时的期望交叉熵损失。
    """
    return -np.log(1.0 / vocab_size) * seq_length


def initialize_parameters(n_a, n_x, n_y):
    """
    用小随机值初始化模型参数矩阵。
    
    参数：
        n_a -- 隐藏层大小（hidden units）
        n_x -- 输入维度（通常等于 vocab_size）
        n_y -- 输出维度（通常等于 vocab_size）
    返回：
        parameters -- 字典，包含以下键：
            Wax (n_a, n_x)   : 输入到隐藏层的权重
            Waa (n_a, n_a)   : 隐藏到隐藏的权重
            Wya (n_y, n_a)   : 隐藏到输出的权重
            b   (n_a, 1)     : 隐藏层偏置
            by  (n_y, 1)     : 输出层偏置
    说明：
        采用小的随机数乘以 0.01 以便训练开始时梯度不会太大。
    """
    np.random.seed(1)  # 固定随机数种子以便示例可复现（教学演示用）
    Wax = np.random.randn(n_a, n_x) * 0.01  # 输入->隐藏
    Waa = np.random.randn(n_a, n_a) * 0.01  # 隐藏->隐藏
    Wya = np.random.randn(n_y, n_a) * 0.01  # 隐藏->输出
    b = np.zeros((n_a, 1))  # 隐藏偏置初始为 0
    by = np.zeros((n_y, 1))  # 输出偏置初始为 0

    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
    return parameters


def rnn_step_forward(parameters, a_prev, x):
    """
    RNN 单时间步前向传播（timestep forward）。
    
    参数：
        parameters -- 字典，包含 Waa, Wax, Wya, by, b
        a_prev -- 上一时间步的隐藏状态，形状 (n_a, m) ，通常 m=1 在逐样本训练时
        x -- 当前时间步的输入独热向量，形状 (vocab_size, m)
    返回：
        a_next -- 本时间步的隐藏状态，形状 (n_a, m)
        p_t -- 本时间步的输出概率分布（softmax），形状 (vocab_size, m)
    
    计算关系：
        a_next = tanh(Wax * x + Waa * a_prev + b)
        p_t = softmax(Wya * a_next + by)
    注：
        tanh 给隐藏层引入非线性，softmax 给输出概率分布，p_t[:, i] 表示第 i 个样本分布。
    """
    # 从参数字典中取出对应矩阵（注意顺序）
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    # 计算下一隐藏状态（使用双曲正切激活）
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)  # shape (n_a, m)
    # 计算输出 logits，并经 softmax 得到概率分布
    p_t = softmax(np.dot(Wya, a_next) + by)  # shape (vocab_size, m)
    return a_next, p_t


def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    """
    RNN 单时间步反向传播（timestep backward）
    
    参数：
        dy -- 当前时间步输出层关于 logits 的梯度（或关于 softmax 输出 y_hat 的 “误差”），形状 (vocab_size, m)
              在交叉熵与 softmax 的组合下，dy = y_hat; dy[y_true] -= 1 （见 rnn_backward）
        gradients -- 累积的梯度字典（会在此函数中被就地更新）
        parameters -- 参数字典（用于访问 Wya, Waa 等）
        x -- 当前时间步的输入独热向量，shape (vocab_size, m)
        a -- 当前时间步的隐藏状态 a[t], shape (n_a, m)
        a_prev -- 前一时间步隐藏状态 a[t-1], shape (n_a, m)
    返回：
        gradients -- 更新后的梯度字典（含 dWax, dWaa, dWya, db, dby, da_next）
    
    反向传播步骤（按链式法则）：
      1) 输出层梯度 -> dWya, dby
      2) 把输出层误差传回隐藏层 -> da
      3) 通过 tanh 导数传播到线性项 -> daraw
      4) 计算 dWax, dWaa, db，最终计算返回 da_next（传给上一时间步）
    """
    # 1) 输出层梯度累加：dWya 和 dby（注意 dy 的 shape）
    gradients['dWya'] += np.dot(dy, a.T)  # (vocab_size, n_a) += (vocab_size,m) * (m, n_a)
    gradients['dby'] += dy                # (vocab_size, 1) 累加每列的 dy（如果 m>1，应 sum）
    # 2) 将输出误差传回隐藏层 a 的方向
    #    Wya^T * dy -> shape (n_a, m)
    #    gradients['da_next'] 是来自下一时间步（t+1）的隐藏层梯度，已在外部累积
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next']  # shape (n_a, m)
    # 3) 通过 tanh 激活的导数传播：
    #    a = tanh(linear), tanh'(z) = 1 - a^2
    daraw = (1 - a * a) * da  # element-wise 乘，shape (n_a, m)
    # 4) 将 daraw 传播到偏置和权重：
    gradients['db'] += daraw  # 隐藏层偏置梯度累加，shape (n_a, 1)
    gradients['dWax'] += np.dot(daraw, x.T)       # (n_a, n_x) += (n_a, m) * (m, n_x)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)  # (n_a, n_a) += (n_a, m) * (m, n_a)
    # 5) 计算传回上一时间步的梯度 da_next（用于传播到 t-1）
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)  # shape (n_a, m)
    return gradients


def update_parameters(parameters, gradients, lr):
    """
    使用简单的梯度下降更新参数（参数就地更新）
    
    参数：
        parameters -- 字典，包含 Wax, Waa, Wya, b, by
        gradients -- 字典，包含对应的梯度 dWax, dWaa, dWya, db, dby
        lr -- 学习率（learning rate）
    返回：
        parameters -- 更新后的参数字典（同一个引用被修改）
    说明：
        参数更新规则：W := W - lr * dW
    """
    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b'] += -lr * gradients['db']
    parameters['by'] += -lr * gradients['dby']
    return parameters


def rnn_forward(X, Y, a0, parameters, vocab_size=71):
    """
    对一个序列 X 做前向传播并计算总损失（交叉熵）。
    
    参数：
        X -- list 或可迭代对象，长度为 T，包含每个时间步的输入索引（整数）
             例如 X[t] = idx (0 <= idx < vocab_size)
        Y -- 目标索引列表，长度与 X 相同（表示每个时间步的真实下一个字符）
        a0 -- 初始隐藏状态，shape (n_a, m)
        parameters -- 参数字典（Wax, Waa, Wya, b, by）
        vocab_size -- 词表大小（用于构造 one-hot 向量的维度），默认 71（示例值，可按需修改）
    返回：
        loss -- 序列的交叉熵损失（标量）
        cache -- 包含前向传播中保存的中间变量 (y_hat, a, x)，用于反向传播
    注意：
        这里实现适合 batch size m=1 或将每列作为一个样本的情形；
        本实现中 x[t] 为 (vocab_size, 1) 的 one-hot 向量。
    """
    # 使用字典来保存每个时间步的 x, a, y_hat，便于反向传播逐步回溯
    x, a, y_hat = {}, {}, {}
    # 把 a[-1] 设为 a0，便于在第 0 步中使用 a[-1] 作为上一步隐藏状态
    a[-1] = np.copy(a0)
    # 初始化损失值为 0
    loss = 0

    # 遍历时间步 t
    for t in range(len(X)):
        # 1) 将 X[t]（整数索引）转为 one-hot 向量 x[t]
        x[t] = np.zeros((vocab_size, 1))
        x[t][X[t]] = 1  # 将对应索引位置置 1

        # 2) 单步前向传播，得到隐藏状态 a[t] 和预测分布 y_hat[t]
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t - 1], x[t])

        # 3) 累加交叉熵损失： -log p(y_true)
        #    y_hat[t] 是概率向量，Y[t] 是真实索引
        loss -= np.log(y_hat[t][Y[t], 0])

    # 保存需要用于反向传播的中间变量
    cache = (y_hat, a, x)
    return loss, cache


def rnn_backward(X, Y, parameters, cache):
    """
    对整个序列做 BPTT（backprop through time），计算并返回参数梯度。
    
    参数：
        X -- 输入索引序列（长度 T）
        Y -- 目标索引序列（长度 T）
        parameters -- 参数字典（用于读取矩阵维度）
        cache -- rnn_forward 返回的缓存 (y_hat, a, x)
    返回：
        gradients -- 字典，包含 dWax, dWaa, dWya, db, dby（对应的累积梯度）
        a -- 隐藏状态字典（可以用于返回最后一层隐藏状态等）
    说明：
        这里使用交叉熵 + softmax 的标准 trick：
        对每个时间步： dy = y_hat.copy(); dy[y_true] -= 1
        然后将 dy 传入 rnn_step_backward。
    """
    # 初始化梯度字典
    gradients = {}

    # 从 cache 中取出中间值
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']

    # 初始化与参数维度相同的零矩阵用于累加梯度
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    # da_next 用于存放下一时间步传回的隐藏层梯度（初始化为零）
    gradients['da_next'] = np.zeros_like(a[0])

    # 反向遍历所有时间步
    for t in reversed(range(len(X))):
        # dy = y_hat[t]，然后在真实标签位置上减 1（交叉熵 softmax 的简化梯度）
        dy = np.copy(y_hat[t])  # shape (vocab_size, m)
        dy[Y[t]] -= 1
        # 调用单步反向传播（内部会更新 gradients 并返回）
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t - 1])

    # 返回累积好的梯度和隐藏状态字典
    return gradients, a


In [82]:
def sigmoid(x):
    """
    计算 sigmoid 激活函数
    
    参数：
        x -- 输入，可以是标量、向量或矩阵（numpy array）
             shape 任意，如 (n, m)
    
    返回：
        s -- 对应 x 的 sigmoid 值，shape 与 x 相同
    
    公式：
        sigmoid(x) = 1 / (1 + exp(-x))
    
    说明：
        1) 输出值在 (0, 1) 之间，常用于概率或门控机制（如 LSTM/GRU 中的遗忘门、输入门、输出门）。
        2) 对于标量 x，返回单个值；对向量或矩阵，按元素计算。
        3) 这是一个逐元素操作，使用 numpy 的广播功能。
    """
    # 使用 numpy 的逐元素 exp 计算
    s = 1 / (1 + np.exp(-x))
    return s


## 1 - 基本循环神经网络（RNN）的前向传播

在本周稍后的部分，你将使用 RNN 来生成音乐。  
你即将实现的基本 RNN 的结构如下图所示。  
在本例中，$T_x = T_y$。


<img src="images/RNN.png" style="width:500;height:300px;">
<caption><center> **Figure 1**: Basic RNN model </center></caption>

下面是实现一个循环神经网络（RNN）的方式：

**步骤**：
1. 实现 RNN 在**单个时间步（time-step）**上的计算。
2. 在所有时间步 $T_x$ 上进行循环，以逐个处理所有输入。

让我们开始吧！

---

## 1.1 - RNN 单元（RNN Cell）

循环神经网络（Recurrent Neural Network）可以看作是**同一个单元在时间上的重复**。  
我们首先要实现单个时间步的计算。下图展示了一个 RNN 单元在单个时间步的计算过程：

<img src="images/rnn_step_forward.png" style="width:700px;height:300px;">
<caption><center> **图 2**：基本 RNN 单元。输入为当前输入 $x^{\langle t \rangle}$ 和前一时刻的隐藏状态 $a^{\langle t - 1\rangle}$（包含过去的信息），输出为新的隐藏状态 $a^{\langle t \rangle}$，它既会传递给下一个 RNN 单元，也会用于预测当前输出 $y^{\langle t \rangle}$。 </center></caption>

---

**练习**：实现图 2 所示的 RNN 单元。

**步骤说明**：
1. 使用 `tanh` 激活函数计算隐藏状态：
   $$
   a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)
   $$
2. 使用新的隐藏状态 $a^{\langle t \rangle}$ 计算预测值：
   $$
   \hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)
   $$
   这里你可以使用提供的 `softmax` 函数。
3. 将以下变量存入缓存（cache）：
   $$
   (a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parameters)
   $$
4. 返回以下结果：
   $$
   a^{\langle t \rangle}, \; y^{\langle t \rangle}, \; \text{cache}
   $$

---

在实现时，我们会对 **$m$ 个样本**进行向量化处理：  
- 输入向量 $x^{\langle t \rangle}$ 的维度为 $(n_x, m)$；  
- 隐藏状态向量 $a^{\langle t \rangle}$ 的维度为 $(n_a, m)$。


In [83]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    实现 RNN 单元在单个时间步的前向传播
    参考图示（例如课程中 Figure 2）

    参数说明:
    xt -- 当前时间步 t 的输入数据，numpy 数组，形状为 (n_x, m)
          n_x 表示输入特征数量，m 表示样本数量（batch size）
    a_prev -- 前一时间步 t-1 的隐藏状态，numpy 数组，形状为 (n_a, m)
              n_a 表示隐藏状态的维度
    parameters -- Python 字典，包含模型的参数：
                        Wax -- 输入权重矩阵，形状 (n_a, n_x)，用于将输入 xt 投影到隐藏状态
                        Waa -- 隐藏状态权重矩阵，形状 (n_a, n_a)，用于将上一个隐藏状态 a_prev 投影到下一隐藏状态
                        Wya -- 输出权重矩阵，形状 (n_y, n_a)，用于将隐藏状态映射到输出
                        ba -- 隐藏状态偏置向量，形状 (n_a, 1)
                        by -- 输出偏置向量，形状 (n_y, 1)
    返回值:
    a_next -- 下一时间步 t 的隐藏状态，形状 (n_a, m)
    yt_pred -- 当前时间步 t 的预测输出，numpy 数组，形状 (n_y, m)
    cache -- 元组，保存前向传播所需的值，用于反向传播，包含 (a_next, a_prev, xt, parameters)
    """
    
    # ------------------1. 从参数字典中提取权重和偏置------------------
    Wax = parameters["Wax"]  # 输入到隐藏状态的权重矩阵 (n_a, n_x)
    Waa = parameters["Waa"]  # 上一隐藏状态到当前隐藏状态的权重矩阵 (n_a, n_a)
    Wya = parameters["Wya"]  # 隐藏状态到输出的权重矩阵 (n_y, n_a)
    ba = parameters["ba"]    # 隐藏状态偏置向量 (n_a, 1)
    by = parameters["by"]    # 输出偏置向量 (n_y, 1)
    
    # ------------------2. 计算下一隐藏状态 a_next ------------------
    # np.dot(A, B) 表示矩阵 A 与矩阵 B 的乘法
    # a_next = tanh(Waa * a_prev + Wax * xt + ba)
    # tanh 是双曲正切函数，作用是将值压缩到 [-1, 1]，增加非线性
    # 公式解释：
    #    Waa * a_prev  -> 上一隐藏状态的影响
    #    Wax * xt      -> 当前输入的影响
    #    ba            -> 偏置项，增加模型表达能力
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    
    # ------------------3. 计算当前时间步的输出 yt_pred ------------------
    # 将隐藏状态映射到输出空间，并使用 softmax 得到预测概率
    # yt_pred = softmax(Wya * a_next + by)
    # softmax 函数将向量转换为概率分布，输出每个类别的概率
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    
    # ------------------4. 保存缓存值，用于反向传播 ------------------
    # cache 中保存了前向传播所需的所有变量
    # 反向传播时可以使用这些值计算梯度
    cache = (a_next, a_prev, xt, parameters)
    
    # 返回下一隐藏状态、当前输出预测和缓存
    return a_next, yt_pred, cache


In [84]:
# ------------------1. 设置随机种子------------------
# np.random.seed(1) 用于固定随机数生成器的种子
# 这样每次生成的随机数都是一样的，保证实验可重复
np.random.seed(1)

# ------------------2. 生成随机输入数据------------------
# xt 是当前时间步 t 的输入数据，形状为 (n_x, m)
# 这里 n_x=3 表示输入特征数量为 3，m=10 表示样本数为 10
# np.random.randn(3,10) 生成标准正态分布（均值=0，方差=1）的随机数组
xt = np.random.randn(3,10)

# a_prev 是前一时间步 t-1 的隐藏状态，形状为 (n_a, m)
# 这里 n_a=5 表示隐藏状态维度为 5，m=10 表示样本数
a_prev = np.random.randn(5,10)

# ------------------3. 生成随机权重和偏置------------------
# Waa：上一个隐藏状态到当前隐藏状态的权重矩阵，形状 (n_a, n_a) = (5,5)
Waa = np.random.randn(5,5)

# Wax：输入到隐藏状态的权重矩阵，形状 (n_a, n_x) = (5,3)
Wax = np.random.randn(5,3)

# Wya：隐藏状态到输出的权重矩阵，形状 (n_y, n_a) = (2,5)
Wya = np.random.randn(2,5)

# ba：隐藏状态偏置，形状 (n_a,1) = (5,1)
ba = np.random.randn(5,1)

# by：输出偏置，形状 (n_y,1) = (2,1)
by = np.random.randn(2,1)

# ------------------4. 将权重和偏置封装到字典中------------------
# parameters 字典中存储了所有 RNN 参数，方便传入函数
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

# ------------------5. 调用 rnn_cell_forward 函数进行单步前向传播------------------
# 输入当前时间步 xt、前一隐藏状态 a_prev、参数字典 parameters
# 返回：
# a_next：下一隐藏状态
# yt_pred：当前时间步预测输出
# cache：前向传播所需的缓存，用于反向传播
a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)

# ------------------6. 打印部分结果进行验证------------------
# 打印 a_next 中第 5 个隐藏单元的所有样本值
print("a_next[4] = ", a_next[4])

# 打印 a_next 的形状，应该是 (n_a, m) = (5,10)
print("a_next.shape = ", a_next.shape)

# 打印 yt_pred 中第 2 个输出单元的所有样本预测概率
print("yt_pred[1] =", yt_pred[1])

# 打印 yt_pred 的形状，应该是 (n_y, m) = (2,10)
print("yt_pred.shape = ", yt_pred.shape)


a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape =  (2, 10)


**Expected Output**: 

<table>
    <tr>
        <td>
            **a_next[4]**:
        </td>
        <td>
           [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
        </td>
    </tr>
        <tr>
        <td>
            **a_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **yt[1]**:
        </td>
        <td>
           [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
        </td>
    </tr>
        <tr>
        <td>
            **yt.shape**:
        </td>
        <td>
           (2, 10)
        </td>
    </tr>

</table>

## 1.2 - RNN 前向传播（Forward Pass）

你可以将循环神经网络（RNN）看作是刚刚实现的 RNN 单元（cell）在时间上的重复。  
如果输入序列包含 10 个时间步（time steps），那么这个 RNN 单元就会被“复制” 10 次。  

每个 RNN 单元都会接收：
- 来自前一时刻的隐藏状态 $a^{\langle t-1 \rangle}$，
- 当前时刻的输入数据 $x^{\langle t \rangle}$。

它会输出：
- 当前时刻的隐藏状态 $a^{\langle t \rangle}$，
- 当前时刻的预测值 $y^{\langle t \rangle}$。

---

<img src="images/rnn.png" style="width:800px;height:300px;">
<caption><center> **图 3**：基本 RNN。输入序列 $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ 经过 $T_x$ 个时间步传播。  
网络输出序列 $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$。 </center></caption>

---

**练习**：实现图 3 所示 RNN 的前向传播过程。

**实现步骤**：
1. 创建一个全零矩阵 `a`，用于存储 RNN 在每个时间步计算得到的隐藏状态。
2. 将初始隐藏状态设为 $a_0$。
3. 开始遍历每一个时间步（索引为 $t$）：
   - 调用 `rnn_cell_forward` 函数，更新“下一时刻”的隐藏状态和缓存（cache）；
   - 将新的隐藏状态 $a^{\langle t \rangle}$ 存入矩阵 `a` 的第 $t$ 个位置；
   - 将预测值 $y^{\langle t \rangle}$ 存入输出矩阵 `y`；
   - 将本时间步的缓存对象添加到缓存列表 `caches` 中。
4. 返回结果：
   $$
   a, \; y, \; \text{caches}
   $$


In [85]:
# 定义函数 rnn_forward，用于计算整个序列的 RNN 前向传播
def rnn_forward(x, a0, parameters):
    """
    实现 RNN 对整个序列的前向传播
    参考图示（例如课程 Figure 3）

    参数说明:
    x -- 所有时间步的输入数据，形状为 (n_x, m, T_x)
         n_x 表示输入特征数量，m 表示样本数量，T_x 表示序列长度（时间步数）
    a0 -- 初始隐藏状态，形状为 (n_a, m)
    parameters -- Python 字典，包含 RNN 参数：
                        Waa -- 隐藏状态权重矩阵，形状 (n_a, n_a)
                        Wax -- 输入权重矩阵，形状 (n_a, n_x)
                        Wya -- 输出权重矩阵，形状 (n_y, n_a)
                        ba -- 隐藏状态偏置，形状 (n_a, 1)
                        by -- 输出偏置，形状 (n_y, 1)

    返回值:
    a -- 所有时间步的隐藏状态，形状 (n_a, m, T_x)
    y_pred -- 所有时间步的预测输出，形状 (n_y, m, T_x)
    caches -- 元组，保存前向传播的缓存值，包含 (list of caches, x)，用于反向传播
    """
    
    # ------------------1. 初始化缓存列表------------------
    # caches 用于存储每个时间步的 cache，后续反向传播会用到
    caches = []
    
    # ------------------2. 获取输入和输出维度------------------
    # x.shape 返回 (n_x, m, T_x)
    n_x, m, T_x = x.shape
    
    # Wya.shape 返回 (n_y, n_a)
    n_y, n_a = parameters["Wya"].shape
    
    # ------------------3. 初始化隐藏状态和输出------------------
    # a 用于存储每个时间步的隐藏状态，形状 (n_a, m, T_x)，全部初始化为 0
    a = np.zeros([n_a, m, T_x])
    
    # y_pred 用于存储每个时间步的输出预测，形状 (n_y, m, T_x)，全部初始化为 0
    y_pred = np.zeros([n_y, m, T_x])
    
    # a_next 用于迭代保存当前时间步的隐藏状态，初始值为 a0
    a_next = a0
    
    # ------------------4. 遍历每个时间步------------------
    for t in range(T_x):
        # 4.1 使用 rnn_cell_forward 函数计算当前时间步的隐藏状态和预测输出
        # x[:, :, t] 表示第 t 个时间步的输入数据，形状 (n_x, m)
        # a_next 表示上一时间步的隐藏状态
        # parameters 为 RNN 参数字典
        a_next, yt_pred, cache = rnn_cell_forward(x[:, :, t], a_next, parameters)
        
        # 4.2 保存当前时间步的隐藏状态到 a 中
        # a[:, :, t] 对应序列中第 t 个时间步的隐藏状态
        a[:, :, t] = a_next
        
        # 4.3 保存当前时间步的输出预测到 y_pred 中
        y_pred[:, :, t] = yt_pred
        
        # 4.4 将当前时间步的 cache 添加到缓存列表 caches 中
        caches.append(cache)
    
    # ------------------5. 保存整个序列前向传播的缓存------------------
    # 最终 caches 保存为一个元组，包含列表 caches 和原始输入 x
    # 方便在反向传播中使用
    caches = (caches, x)
    
    # 返回整个序列的隐藏状态、预测输出和缓存
    return a, y_pred, caches


In [86]:
# ------------------1. 固定随机种子------------------
# np.random.seed(1) 保证每次生成的随机数相同，实验可重复
np.random.seed(1)

# ------------------2. 生成随机序列输入------------------
# x 是整个序列的输入数据，形状为 (n_x, m, T_x)
# n_x = 3 表示每个时间步输入特征数
# m = 10 表示样本数（batch size）
# T_x = 4 表示序列长度（时间步数）
# np.random.randn(3,10,4) 生成标准正态分布的随机数
x = np.random.randn(3,10,4)

# a0 是初始隐藏状态，形状为 (n_a, m)
# n_a = 5 表示隐藏状态维度，m = 10 表示样本数
a0 = np.random.randn(5,10)

# ------------------3. 生成随机权重和偏置------------------
# Waa：隐藏状态权重矩阵，形状 (n_a, n_a) = (5,5)
Waa = np.random.randn(5,5)

# Wax：输入到隐藏状态的权重矩阵，形状 (n_a, n_x) = (5,3)
Wax = np.random.randn(5,3)

# Wya：隐藏状态到输出的权重矩阵，形状 (n_y, n_a) = (2,5)
Wya = np.random.randn(2,5)

# ba：隐藏状态偏置，形状 (n_a,1) = (5,1)
ba = np.random.randn(5,1)

# by：输出偏置，形状 (n_y,1) = (2,1)
by = np.random.randn(2,1)

# ------------------4. 封装参数字典------------------
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

# ------------------5. 调用 rnn_forward 函数进行序列前向传播------------------
# 输入整个序列 x、初始隐藏状态 a0、参数字典 parameters
# 返回：
# a：整个序列的隐藏状态，形状 (n_a, m, T_x)
# y_pred：整个序列的预测输出，形状 (n_y, m, T_x)
# caches：前向传播缓存，用于反向传播
a, y_pred, caches = rnn_forward(x, a0, parameters)

# ------------------6. 打印部分结果进行验证------------------
# 打印隐藏状态 a 中第 5 个隐藏单元、第 2 个样本在所有时间步的位置
print("a[4][1] = ", a[4][1])
# 打印隐藏状态 a 的形状，应该为 (n_a, m, T_x) = (5, 10, 4)
print("a.shape = ", a.shape)

# 打印预测输出 y_pred 中第 2 个输出单元、第 4 个时间步在所有样本的位置
print("y_pred[1][3] =", y_pred[1][3])
# 打印预测输出 y_pred 的形状，应该为 (n_y, m, T_x) = (2, 10, 4)
print("y_pred.shape = ", y_pred.shape)

# 打印 caches 中原始输入 x 的某个值
# caches = (list of caches, x)
# caches[1] 是原始输入 x，形状为 (n_x, m, T_x)
# caches[1][1][3] 表示 x 中第 2 个特征、第 2 个样本、第 4 个时间步的值
print("caches[1][1][3] =", caches[1][1][3])

# 打印 caches 的长度
# caches 是一个元组，包含 (list of caches, x)，所以 len(caches) = 2
print("len(caches) = ", len(caches))


a[4][1] =  [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape =  (5, 10, 4)
y_pred[1][3] = [0.79560373 0.86224861 0.11118257 0.81515947]
y_pred.shape =  (2, 10, 4)
caches[1][1][3] = [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) =  2


**Expected Output**:

<table>
    <tr>
        <td>
            **a[4][1]**:
        </td>
        <td>
           [-0.99999375  0.77911235 -0.99861469 -0.99833267]
        </td>
    </tr>
        <tr>
        <td>
            **a.shape**:
        </td>
        <td>
           (5, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **y[1][3]**:
        </td>
        <td>
           [ 0.79560373  0.86224861  0.11118257  0.81515947]
        </td>
    </tr>
        <tr>
        <td>
            **y.shape**:
        </td>
        <td>
           (2, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **cache[1][1][3]**:
        </td>
        <td>
           [-1.1425182  -0.34934272 -0.20889423  0.58662319]
        </td>
    </tr>
        <tr>
        <td>
            **len(cache)**:
        </td>
        <td>
           2
        </td>
    </tr>

</table>

恭喜你！🎉  
你已经成功地从零实现了一个循环神经网络（RNN）的前向传播过程。  

这个模型在某些应用中表现良好，但它存在一个常见的问题 —— **梯度消失（vanishing gradient）**。  
因此，它在以下情况下表现最佳：  
当每个输出 $y^{\langle t \rangle}$ 主要依赖于**局部上下文信息**时（也就是说，模型只需关注与当前时间步 $t$ 相距不远的输入 $x^{\langle t' \rangle}$）。

---

在接下来的部分中，你将构建一个更复杂的 **LSTM（长短期记忆网络）** 模型。  
与基本 RNN 不同，LSTM 能够更有效地应对梯度消失问题。  
它能更好地“记住”信息，并在多个时间步中持续保留这些记忆。


## 2 - 长短期记忆网络（LSTM）

下图展示了 LSTM 单元（cell）的操作流程：

<img src="images/LSTM.png" style="width:500;height:400px;">
<caption><center> **图 4**：LSTM 单元。每个时间步都会追踪并更新一个“细胞状态”或记忆变量 $c^{\langle t \rangle}$，它可能与隐藏状态 $a^{\langle t \rangle}$ 不同。 </center></caption>

---

类似于上面的 RNN 示例，你将先实现单个时间步的 LSTM 单元。  
然后，可以在 `for` 循环中迭代调用该单元，以处理长度为 $T_x$ 的输入序列。

---

### 关于门（Gates）

#### - 遗忘门（Forget Gate）

为了便于理解，假设我们正在阅读一段文本，希望用 LSTM 跟踪语法结构，例如判断主语是单数还是复数。  
如果主语从单数变为复数，我们需要找到一种方法**丢弃之前存储的单数/复数状态信息**。  
在 LSTM 中，遗忘门可以实现这一功能：

$$
\Gamma_f^{\langle t \rangle} = \sigma(W_f[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_f)\tag{1} 
$$

其中，$W_f$ 是控制遗忘门行为的权重。我们将 $[a^{\langle t-1 \rangle}, x^{\langle t \rangle}]$ 拼接后乘以 $W_f$，得到 $\Gamma_f^{\langle t \rangle}$，其值在 0 到 1 之间。  
这个遗忘门向量会与前一时刻的细胞状态 $c^{\langle t-1 \rangle}$ **逐元素相乘**。  
- 如果 $\Gamma_f^{\langle t \rangle}$ 的某个值为 0（或接近 0），表示 LSTM 应该删除对应的记忆信息（如单数主语）；  
- 如果值为 1，则保留该信息。

---

#### - 更新门（Update Gate）

在丢弃了旧的单数主语信息后，我们需要更新细胞状态以反映新的复数主语。更新门的公式为：

$$
\Gamma_u^{\langle t \rangle} = \sigma(W_u[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_u)\tag{2} 
$$

类似于遗忘门，$\Gamma_u^{\langle t \rangle}$ 是一个值在 0 到 1 之间的向量。  
它会与候选细胞状态 $\tilde{c}^{\langle t \rangle}$ **逐元素相乘**，用于计算新的细胞状态 $c^{\langle t \rangle}$。

---

#### - 更新细胞状态

为了更新新的主语信息，我们需要创建一个新的数值向量，加入到之前的细胞状态中：

$$
\tilde{c}^{\langle t \rangle} = \tanh(W_c[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_c)\tag{3} 
$$

最终新的细胞状态为：

$$
c^{\langle t \rangle} = \Gamma_f^{\langle t \rangle} * c^{\langle t-1 \rangle} + \Gamma_u^{\langle t \rangle} * \tilde{c}^{\langle t \rangle} \tag{4} 
$$

---

#### - 输出门（Output Gate）

决定输出时使用以下公式：

$$
\Gamma_o^{\langle t \rangle}=  \sigma(W_o[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_o)\tag{5}
$$
$$
a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle} * \tanh(c^{\langle t \rangle})\tag{6} 
$$

公式 (5) 使用 sigmoid 函数决定输出的权重，公式 (6) 再将其与当前细胞状态的 $\tanh$ 值逐元素相乘，得到最终隐藏状态。


### 2.1 - LSTM 单元（LSTM Cell）

**练习**：实现图 3 中所示的 LSTM 单元。

**实现步骤**：
1. 将 $a^{\langle t-1 \rangle}$ 和 $x^{\langle t \rangle}$ 拼接成一个矩阵：
   $$
   concat = \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}
   $$
2. 计算公式 (1) 至 (6) 中的各个门和状态。  
   - 可以使用提供的 `sigmoid()` 函数计算遗忘门、更新门和输出门；  
   - 使用 `np.tanh()` 计算候选细胞状态和隐藏状态。
3. 计算预测值 $y^{\langle t \rangle}$，可以使用提供的 `softmax()` 函数。


In [87]:
# 定义函数 lstm_cell_forward，用于计算 LSTM 单元在单个时间步的前向传播
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    实现 LSTM 单元在单个时间步的前向传播
    参考图示（例如课程 Figure 4）

    参数说明:
    xt -- 当前时间步 t 的输入数据，形状为 (n_x, m)
    a_prev -- 前一时间步 t-1 的隐藏状态，形状为 (n_a, m)
    c_prev -- 前一时间步 t-1 的记忆状态，形状为 (n_a, m)
    parameters -- Python 字典，包含 LSTM 参数：
                        Wf -- 遗忘门权重矩阵，形状 (n_a, n_a + n_x)
                        bf -- 遗忘门偏置，形状 (n_a, 1)
                        Wi -- 更新门（输入门）权重矩阵，形状 (n_a, n_a + n_x)
                        bi -- 更新门偏置，形状 (n_a, 1)
                        Wc -- 候选记忆单元权重矩阵，形状 (n_a, n_a + n_x)
                        bc -- 候选记忆单元偏置，形状 (n_a, 1)
                        Wo -- 输出门权重矩阵，形状 (n_a, n_a + n_x)
                        bo -- 输出门偏置，形状 (n_a, 1)
                        Wy -- 输出权重矩阵，形状 (n_y, n_a)
                        by -- 输出偏置，形状 (n_y, 1)

    返回值:
    a_next -- 下一隐藏状态，形状 (n_a, m)
    c_next -- 下一记忆状态，形状 (n_a, m)
    yt_pred -- 当前时间步预测输出，形状 (n_y, m)
    cache -- 元组，保存前向传播所需的变量，用于反向传播
             包含 (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)
    """

    # ------------------1. 从参数字典中提取权重和偏置------------------
    Wf = parameters["Wf"]  # 遗忘门权重
    bf = parameters["bf"]  # 遗忘门偏置
    Wi = parameters["Wi"]  # 更新门权重
    bi = parameters["bi"]  # 更新门偏置
    Wc = parameters["Wc"]  # 候选记忆单元权重
    bc = parameters["bc"]  # 候选记忆单元偏置
    Wo = parameters["Wo"]  # 输出门权重
    bo = parameters["bo"]  # 输出门偏置
    Wy = parameters["Wy"]  # 输出权重
    by = parameters["by"]  # 输出偏置

    # ------------------2. 获取输入和输出维度------------------
    n_x, m = xt.shape       # n_x: 输入特征数，m: 样本数量
    n_y, n_a = Wy.shape     # n_y: 输出维度，n_a: 隐藏状态维度

    # ------------------3. 连接 a_prev 与 xt------------------
    # LSTM 的门操作同时依赖上一隐藏状态 a_prev 和当前输入 xt
    # 创建一个 (n_a + n_x, m) 的矩阵 contact，将 a_prev 和 xt 竖直拼接
    contact = np.zeros([n_a + n_x, m])
    contact[: n_a, :] = a_prev  # 上半部分为 a_prev
    contact[n_a :, :] = xt      # 下半部分为 xt

    # ------------------4. 计算各个门和候选记忆单元------------------

    ## 遗忘门 ft，公式1
    # ft = sigmoid(Wf * contact + bf)
    # 控制上一记忆状态 c_prev 中的信息保留程度
    ft = sigmoid(np.dot(Wf, contact) + bf)

    ## 更新门 it，公式2
    # it = sigmoid(Wi * contact + bi)
    # 控制候选记忆单元信息对 c_next 的贡献比例
    it = sigmoid(np.dot(Wi, contact) + bi)

    ## 候选记忆单元 cct，公式3
    # cct = tanh(Wc * contact + bc)
    # 生成当前时间步候选更新信息，范围 [-1,1]
    cct = np.tanh(np.dot(Wc, contact) + bc)

    ## 计算下一记忆状态 c_next，公式4
    # c_next = ft * c_prev + it * cct
    # 遗忘门保留上一记忆，更新门引入新候选记忆
    c_next = ft * c_prev + it * cct

    ## 输出门 ot，公式5
    # ot = sigmoid(Wo * contact + bo)
    # 控制当前隐藏状态输出的比例
    ot = sigmoid(np.dot(Wo, contact) + bo)

    ## 计算下一隐藏状态 a_next，公式6
    # a_next = ot * tanh(c_next)
    # 将记忆状态通过 tanh 变换后，再由输出门调节
    a_next = ot * np.tanh(c_next)

    # ------------------5. 计算当前时间步的预测输出------------------
    # yt_pred = softmax(Wy * a_next + by)
    # 将隐藏状态映射到输出空间，并使用 softmax 得到概率分布
    yt_pred = softmax(np.dot(Wy, a_next) + by)

    # ------------------6. 保存缓存，用于反向传播------------------
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    # 返回下一隐藏状态、下一记忆状态、预测输出和缓存
    return a_next, c_next, yt_pred, cache


In [88]:
# ------------------1. 固定随机种子------------------
# np.random.seed(1) 用于保证每次生成的随机数相同，方便实验复现
np.random.seed(1)

# ------------------2. 生成随机输入数据------------------
# xt: 当前时间步 t 的输入，形状 (n_x, m) = (3, 10)
# n_x = 3 输入特征数，m = 10 样本数
xt = np.random.randn(3,10)

# a_prev: 前一时间步 t-1 的隐藏状态，形状 (n_a, m) = (5, 10)
a_prev = np.random.randn(5,10)

# c_prev: 前一时间步 t-1 的记忆状态，形状 (n_a, m) = (5, 10)
c_prev = np.random.randn(5,10)

# ------------------3. 生成随机权重和偏置------------------
# LSTM 有多个门，每个门有权重矩阵和偏置向量
# Wf, bf: 遗忘门权重和偏置，形状 (n_a, n_a + n_x) 和 (n_a, 1)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)

# Wi, bi: 更新门（输入门）权重和偏置
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)

# Wo, bo: 输出门权重和偏置
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)

# Wc, bc: 候选记忆单元权重和偏置
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)

# Wy, by: 输出预测的权重和偏置
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

# ------------------4. 封装参数字典------------------
parameters = {
    "Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc,
    "Wy": Wy,
    "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by
}

# ------------------5. 调用 lstm_cell_forward 函数进行单步前向传播------------------
# 输入当前时间步 xt、前一隐藏状态 a_prev、前一记忆状态 c_prev、参数字典 parameters
# 返回：
# a_next: 下一隐藏状态 (n_a, m)
# c_next: 下一记忆状态 (n_a, m)
# yt: 当前时间步预测输出 (n_y, m)
# cache: 前向传播缓存，用于反向传播
a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)

# ------------------6. 打印部分结果进行验证------------------
# 打印 a_next 中第 5 个隐藏单元的所有样本值
print("a_next[4] = ", a_next[4])

# 打印 a_next 的形状，应该为 (n_a, m) = (5, 10)
print("a_next.shape = ", a_next.shape)

# 打印 c_next 中第 3 个记忆单元的所有样本值
print("c_next[2] = ", c_next[2])

# 打印 c_next 的形状，应该为 (n_a, m) = (5, 10)
print("c_next.shape = ", c_next.shape)

# 打印 yt 中第 2 个输出单元的所有样本预测概率
print("yt[1] =", yt[1])

# 打印 yt 的形状，应该为 (n_y, m) = (2, 10)
print("yt.shape = ", yt.shape)

# 打印 cache 中某个值
# cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)
# cache[1] 表示 c_next，cache[1][3] 表示 c_next 第 4 个样本值
print("cache[1][3] =", cache[1][3])

# 打印 cache 的长度
# cache 是一个元组，包含 a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters
# 长度应该为 10
print("len(cache) = ", len(cache))


a_next[4] =  [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] =  [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [0.79913913 0.15986619 0.22412122 0.15606108 0.97057211 0.31146381
 0.00943007 0.12666353 0.39380172 0.07828381]
yt.shape =  (2, 10)
cache[1][3] = [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10


**Expected Output**:

<table>
    <tr>
        <td>
            **a_next[4]**:
        </td>
        <td>
           [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
        </td>
    </tr>
        <tr>
        <td>
            **a_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **c_next[2]**:
        </td>
        <td>
           [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
        </td>
    </tr>
        <tr>
        <td>
            **c_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **yt[1]**:
        </td>
        <td>
           [ 0.79913913  0.15986619  0.22412122  0.15606108  0.97057211  0.31146381
  0.00943007  0.12666353  0.39380172  0.07828381]
        </td>
    </tr>
        <tr>
        <td>
            **yt.shape**:
        </td>
        <td>
           (2, 10)
        </td>
    </tr>
    <tr>
        <td>
            **cache[1][3]**:
        </td>
        <td>
           [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
        </td>
    </tr>
        <tr>
        <td>
            **len(cache)**:
        </td>
        <td>
           10
        </td>
    </tr>

</table>

### 2.2 - LSTM 前向传播（Forward Pass）

在实现了 LSTM 的单步计算后，你可以通过 `for` 循环将其迭代应用于长度为 $T_x$ 的输入序列。

<img src="images/LSTM_rnn.png" style="width:500;height:300px;">
<caption><center> **图 4**：LSTM 在多个时间步上的展开。 </center></caption>

---

**练习**：实现 `lstm_forward()` 函数，使 LSTM 能够处理 $T_x$ 个时间步的输入序列。

**注意**：初始细胞状态 $c^{\langle 0 \rangle}$ 用全零向量初始化。


In [89]:
# 定义函数 lstm_forward，用于计算整个序列的 LSTM 前向传播
def lstm_forward(x, a0, parameters):
    """
    实现使用 LSTM 单元的 RNN 对整个序列的前向传播
    参考图示（例如课程 Figure 3）

    参数说明:
    x -- 所有时间步的输入数据，形状 (n_x, m, T_x)
         n_x: 输入特征数, m: 样本数, T_x: 序列长度（时间步数）
    a0 -- 初始隐藏状态，形状 (n_a, m)
    parameters -- Python 字典，包含 LSTM 参数：
                        Wf, bf -- 遗忘门权重和偏置
                        Wi, bi -- 更新门权重和偏置
                        Wc, bc -- 候选记忆单元权重和偏置
                        Wo, bo -- 输出门权重和偏置
                        Wy, by -- 输出权重和偏置

    返回值:
    a -- 所有时间步的隐藏状态，形状 (n_a, m, T_x)
    y -- 所有时间步的预测输出，形状 (n_y, m, T_x)
    c -- 所有时间步的记忆状态，形状 (n_a, m, T_x)
    caches -- 元组，包含每个时间步的 cache 列表和原始输入 x，用于反向传播
    """

    # ------------------1. 初始化缓存列表------------------
    # caches 用于存储每个时间步的 cache，后续反向传播会用到
    caches = []

    # ------------------2. 获取输入和输出维度------------------
    # x.shape 返回 (n_x, m, T_x)
    n_x, m, T_x = x.shape
    # Wy.shape 返回 (n_y, n_a)
    n_y, n_a = parameters["Wy"].shape

    # ------------------3. 初始化隐藏状态、记忆状态和预测输出------------------
    # a 用于存储每个时间步的隐藏状态，形状 (n_a, m, T_x)，初始化为 0
    a = np.zeros([n_a, m, T_x])
    # c 用于存储每个时间步的记忆状态，形状 (n_a, m, T_x)，初始化为 0
    c = np.zeros([n_a, m, T_x])
    # y 用于存储每个时间步的预测输出，形状 (n_y, m, T_x)，初始化为 0
    y = np.zeros([n_y, m, T_x])

    # a_next 初始化为 a0，表示第一个时间步的前一隐藏状态
    a_next = a0
    # c_next 初始化为全零矩阵，表示第一个时间步的前一记忆状态
    c_next = np.zeros([n_a, m])

    # ------------------4. 遍历每个时间步------------------
    for t in range(T_x):
        # 调用 lstm_cell_forward，计算当前时间步的：
        # a_next: 下一隐藏状态
        # c_next: 下一记忆状态
        # yt_pred: 当前时间步预测输出
        # cache: 当前时间步的缓存，用于反向传播
        a_next, c_next, yt_pred, cache = lstm_cell_forward(x[:,:,t], a_next, c_next, parameters)

        # ------------------5. 保存当前时间步结果------------------
        # 保存当前时间步的隐藏状态到 a 中
        a[:, :, t] = a_next
        # 保存当前时间步的预测输出到 y 中
        y[:, :, t] = yt_pred
        # 保存当前时间步的记忆状态到 c 中
        c[:, :, t] = c_next
        # 将当前时间步的 cache 添加到缓存列表 caches 中
        caches.append(cache)

    # ------------------6. 保存整个序列前向传播的缓存------------------
    # 最终 caches 保存为一个元组，包含列表 caches 和原始输入 x
    # 方便在反向传播中使用
    caches = (caches, x)

    # 返回整个序列的隐藏状态、预测输出、记忆状态和缓存
    return a, y, c, caches


In [90]:
# ------------------1. 固定随机种子------------------
# 保证每次生成的随机数相同，方便复现实验结果
np.random.seed(1)

# ------------------2. 生成随机序列输入数据------------------
# x: 输入序列数据，形状 (n_x, m, T_x) = (3, 10, 7)
# n_x = 3 输入特征数
# m = 10 样本数量（batch size）
# T_x = 7 序列长度（时间步数）
x = np.random.randn(3,10,7)

# a0: 初始隐藏状态，形状 (n_a, m) = (5, 10)
a0 = np.random.randn(5,10)

# ------------------3. 生成随机 LSTM 权重和偏置------------------
# 遗忘门
Wf = np.random.randn(5, 5+3)  # 权重，形状 (n_a, n_a + n_x)
bf = np.random.randn(5,1)     # 偏置，形状 (n_a, 1)

# 更新门
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)

# 输出门
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)

# 候选记忆单元
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)

# 输出预测权重与偏置
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

# ------------------4. 封装参数字典------------------
parameters = {
    "Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy,
    "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by
}

# ------------------5. 调用 lstm_forward 函数进行整个序列前向传播------------------
# a: 所有时间步的隐藏状态，形状 (n_a, m, T_x) = (5, 10, 7)
# y: 所有时间步的预测输出，形状 (n_y, m, T_x) = (2, 10, 7)
# c: 所有时间步的记忆状态，形状 (n_a, m, T_x) = (5, 10, 7)
# caches: 前向传播缓存，用于反向传播
a, y, c, caches = lstm_forward(x, a0, parameters)

# ------------------6. 打印部分结果进行验证------------------
# 打印隐藏状态 a 中第 5 个隐藏单元、第 4 个样本、第 7 个时间步的值
print("a[4][3][6] = ", a[4][3][6])

# 打印隐藏状态 a 的形状
print("a.shape = ", a.shape)

# 打印预测输出 y 中第 2 个输出单元、第 5 个样本、第 4 个时间步的值
print("y[1][4][3] =", y[1][4][3])

# 打印预测输出 y 的形状
print("y.shape = ", y.shape)

# 打印 caches 中原始输入 x 的某个值
# caches = (list of caches, x)
# caches[1] = x
# caches[1][1][1] 表示 x 中第 2 个特征、第 2 个样本、第 2 个时间步
print("caches[1][1][1] =", caches[1][1][1])

# 打印记忆状态 c 中第 2 个记忆单元、第 3 个样本、第 2 个时间步的值
print("c[1][2][1] =", c[1][2][1])

# 打印 caches 的长度
# caches 是一个元组，包含 (list of caches, x)，所以长度为 2
print("len(caches) = ", len(caches))


a[4][3][6] =  0.17211776753291663
a.shape =  (5, 10, 7)
y[1][4][3] = 0.9508734618501101
y.shape =  (2, 10, 7)
caches[1][1][1] = [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] = -0.8555449167181983
len(caches) =  2


**Expected Output**:


<table>
    <tr>
        <td>
            **a[4][3][6]** =
        </td>
        <td>
           0.172117767533
        </td>
    </tr>
        <tr>
        <td>
            **a.shape** =
        </td>
        <td>
           (5, 10, 7)
        </td>
    </tr>
        <tr>
        <td>
            **y[1][4][3]** =
        </td>
        <td>
           0.95087346185
        </td>
    </tr>
        <tr>
        <td>
            **y.shape** =
        </td>
        <td>
           (2, 10, 7)
        </td>
    </tr>
        <tr>
        <td>
            **caches[1][1][1]** =
        </td>
        <td>
           [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
        </td>
        
     </tr>
        <tr>
        <td>
            **c[1][2][1]** =
        </td>
        <td>
           -0.855544916718
        </td>
    </tr>       
        
    </tr>
        <tr>
        <td>
            **len(caches)** =
        </td>
        <td>
           2
        </td>
    </tr>

</table>

恭喜你！🎉  
你现在已经实现了基本 RNN 和 LSTM 的前向传播（forward pass）。  

在使用深度学习框架时，仅实现前向传播就足以构建高性能的系统。


## 3 - 循环神经网络的反向传播

在现代深度学习框架中，你通常只需要实现前向传播，框架会自动处理反向传播（backward pass）。  
因此，大多数深度学习工程师不需要关心反向传播的具体细节。  
然而，如果你精通微积分并想了解 RNN 的反向传播过程，可以学习本部分。

---

在之前的课程中，你实现了一个简单的（全连接）神经网络，使用反向传播计算损失函数的导数以更新参数。  
同样地，在循环神经网络中，你也可以计算损失函数关于各参数的导数，以进行参数更新。  

反向传播的公式比较复杂，课堂上没有推导，但我们将在下面简要介绍。


### 3.1 - 基本 RNN 的反向传播（Backward Pass）

我们将从基本 RNN 单元的反向传播开始。

<img src="images/rnn_cell_backprop.png" style="width:500;height:300px;"> <br>
<caption><center> **图 5**：RNN 单元的反向传播。  
与全连接神经网络类似，损失函数 $J$ 的导数通过链式法则在 RNN 中进行反向传播。  
链式法则同样用于计算参数的梯度：
$$
\left(\frac{\partial J}{\partial W_{ax}}, \frac{\partial J}{\partial W_{aa}}, \frac{\partial J}{\partial b_a}\right)
$$
以便更新参数 $(W_{ax}, W_{aa}, b_a)$。 </center></caption>


#### 推导单步反向传播函数（One Step Backward）

要实现 `rnn_cell_backward`，需要计算以下公式。  
手动推导这些公式是一个很好的练习。

---

- $\tanh$ 的导数为：
$$
\frac{d}{dx} \tanh(x) = 1 - \tanh(x)^2
$$
完整证明可见 [这里](https://www.wyzant.com/resources/lessons/math/calculus/derivative_proofs/tanx)。  
注意：$\text{sech}(x)^2 = 1 - \tanh(x)^2$。

- 对于以下参数的梯度：
$$
\frac{\partial a^{\langle t \rangle}}{\partial W_{ax}}, \quad \frac{\partial a^{\langle t \rangle}}{\partial W_{aa}}, \quad \frac{\partial a^{\langle t \rangle}}{\partial b}
$$
可以使用 $\tanh(u)$ 的导数公式：
$$
d(\tanh(u)) = (1 - \tanh(u)^2) \, du
$$

- 最后的两个公式也遵循相同的规则，并使用 $\tanh$ 的导数推导而来。  
  注意变量排列方式是为了保证矩阵维度匹配。


In [91]:
# ------------------RNN 单步反向传播函数------------------
# 定义函数 rnn_cell_backward，用于计算 RNN 单个时间步的梯度
def rnn_cell_backward(da_next, cache):
    """
    实现 RNN 单元在单个时间步的反向传播

    参数说明:
    da_next -- 损失函数关于下一隐藏状态 a_next 的梯度，形状 (n_a, m)
    cache -- 来自 rnn_cell_forward 的缓存，包含 (a_next, a_prev, xt, parameters)

    返回值:
    gradients -- 字典，包含各参数的梯度：
                        dx -- 输入 xt 的梯度，形状 (n_x, m)
                        da_prev -- 前一隐藏状态 a_prev 的梯度，形状 (n_a, m)
                        dWax -- 输入到隐藏权重的梯度，形状 (n_a, n_x)
                        dWaa -- 隐藏到隐藏权重的梯度，形状 (n_a, n_a)
                        dba -- 偏置梯度，形状 (n_a, 1)
    """

    # ------------------1. 从缓存中取出前向传播保存的值------------------
    # a_next: 下一隐藏状态
    # a_prev: 前一隐藏状态
    # xt: 当前时间步输入
    # parameters: 参数字典
    (a_next, a_prev, xt, parameters) = cache

    # ------------------2. 从参数字典中提取权重和偏置------------------
    Wax = parameters["Wax"]  # 输入到隐藏的权重
    Waa = parameters["Waa"]  # 隐藏到隐藏的权重
    Wya = parameters["Wya"]  # 隐藏到输出的权重（未在本函数使用）
    ba = parameters["ba"]    # 偏置
    by = parameters["by"]    # 输出偏置（未在本函数使用）

    # ------------------3. 计算 tanh 对 a_next 的梯度------------------
    # a_next = tanh(Waa * a_prev + Wax * xt + ba)
    # tanh 的导数为 (1 - a_next^2)
    # da_next 为损失函数对 a_next 的梯度
    dtanh = (1 - np.square(a_next)) * da_next
    # dtanh: 损失对线性组合 z 的梯度，形状 (n_a, m)

    # ------------------4. 计算输入 xt 的梯度------------------
    # dx = dtanh * Wax.T
    # 使用矩阵乘法将梯度传播到输入 xt
    dxt = np.dot(Wax.T, dtanh)

    # ------------------5. 计算输入到隐藏权重 Wax 的梯度------------------
    # dWax = dtanh * xt.T
    dWax = np.dot(dtanh, xt.T)

    # ------------------6. 计算前一隐藏状态 a_prev 的梯度------------------
    # da_prev = dtanh * Waa.T
    da_prev = np.dot(Waa.T, dtanh)

    # ------------------7. 计算隐藏到隐藏权重 Waa 的梯度------------------
    # dWaa = dtanh * a_prev.T
    dWaa = np.dot(dtanh, a_prev.T)

    # ------------------8. 计算偏置 b 的梯度------------------
    # 对所有样本求和，得到偏置梯度
    dba = np.sum(dtanh, axis=-1, keepdims=True)

    # ------------------9. 将梯度封装到字典中------------------
    gradients = {
        "dxt": dxt,
        "da_prev": da_prev,
        "dWax": dWax,
        "dWaa": dWaa,
        "dba": dba
    }

    # 返回所有梯度
    return gradients


In [92]:
# ------------------1. 固定随机种子------------------
# 保证每次生成的随机数相同，便于实验复现
np.random.seed(1)

# ------------------2. 生成随机输入数据------------------
# xt: 当前时间步输入，形状 (n_x, m) = (3, 10)
xt = np.random.randn(3,10)

# a_prev: 前一时间步隐藏状态，形状 (n_a, m) = (5, 10)
a_prev = np.random.randn(5,10)

# ------------------3. 生成随机权重和偏置------------------
# 输入到隐藏权重 Wax，形状 (n_a, n_x) = (5,3)
Wax = np.random.randn(5,3)

# 隐藏到隐藏权重 Waa，形状 (n_a, n_a) = (5,5)
Waa = np.random.randn(5,5)

# 隐藏到输出权重 Wya，形状 (n_y, n_a) = (2,5)
Wya = np.random.randn(2,5)

# 偏置向量 b (隐藏层) 和 by (输出层)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)

# 封装参数字典
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}

# ------------------4. 调用 rnn_cell_forward 计算前向传播------------------
# a_next: 下一隐藏状态 (n_a, m)
# yt: 当前时间步预测输出 (n_y, m)
# cache: 保存前向传播中间值，用于反向传播
a_next, yt, cache = rnn_cell_forward(xt, a_prev, parameters)

# ------------------5. 生成随机下一隐藏状态梯度------------------
# da_next: 损失函数对下一隐藏状态 a_next 的梯度
da_next = np.random.randn(5,10)

# ------------------6. 调用 rnn_cell_backward 计算梯度------------------
# gradients: 字典，包含各梯度
# dx: 当前输入梯度
# da_prev: 前一隐藏状态梯度
# dWax, dWaa, dba: 权重和偏置梯度
gradients = rnn_cell_backward(da_next, cache)

# ------------------7. 打印部分梯度结果以验证------------------
# 打印输入梯度 dxt 的部分元素
print("gradients[\"dxt\"][1][2] =", gradients["dxt"][1][2])
# 打印输入梯度 dxt 的形状
print("gradients[\"dxt\"].shape =", gradients["dxt"].shape)

# 打印前一隐藏状态梯度 da_prev 的部分元素
print("gradients[\"da_prev\"][2][3] =", gradients["da_prev"][2][3])
# 打印 da_prev 的形状
print("gradients[\"da_prev\"].shape =", gradients["da_prev"].shape)

# 打印输入到隐藏权重梯度 dWax 的部分元素
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
# 打印 dWax 的形状
print("gradients[\"dWax\"].shape =", gradients["dWax"].shape)

# 打印隐藏到隐藏权重梯度 dWaa 的部分元素
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
# 打印 dWaa 的形状
print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape)

# 打印偏置梯度 dba 的部分元素
print("gradients[\"dba\"][4] =", gradients["dba"][4])
# 打印 dba 的形状
print("gradients[\"dba\"].shape =", gradients["dba"].shape)


gradients["dxt"][1][2] = -1.3872130506020925
gradients["dxt"].shape = (3, 10)
gradients["da_prev"][2][3] = -0.1523994937739543
gradients["da_prev"].shape = (5, 10)
gradients["dWax"][3][1] = 0.41077282493545836
gradients["dWax"].shape = (5, 3)
gradients["dWaa"][1][2] = 1.1503450668497135
gradients["dWaa"].shape = (5, 5)
gradients["dba"][4] = [0.20023491]
gradients["dba"].shape = (5, 1)


**Expected Output**:

<table>
    <tr>
        <td>
            **gradients["dxt"][1][2]** =
        </td>
        <td>
           -0.460564103059
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dxt"].shape** =
        </td>
        <td>
           (3, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"][2][3]** =
        </td>
        <td>
           0.0842968653807
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWax"][3][1]** =
        </td>
        <td>
           0.393081873922
        </td>
    </tr>
            <tr>
        <td>
            **gradients["dWax"].shape** =
        </td>
        <td>
           (5, 3)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"][1][2]** = 
        </td>
        <td>
           -0.28483955787
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"].shape** =
        </td>
        <td>
           (5, 5)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"][4]** = 
        </td>
        <td>
           [ 0.80517166]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

#### RNN 的反向传播（Backward Pass）

在每个时间步 $t$ 计算损失函数关于隐藏状态 $a^{\langle t \rangle}$ 的梯度非常重要，  
因为它帮助梯度向前一个 RNN 单元反向传播。  

为此，你需要从序列末尾开始，**倒序遍历所有时间步**，在每个时间步中：
- 累加更新 $db_a, dW_{aa}, dW_{ax}$；
- 保存输入的梯度 $dx$。

---

**实现步骤**：

实现 `rnn_backward` 函数：
1. 首先将返回变量初始化为全零矩阵；
2. 通过倒序循环遍历每个时间步：
   - 调用 `rnn_cell_backward` 函数；
   - 根据计算结果更新相应的梯度变量。


In [93]:
# ------------------RNN 整个序列反向传播函数------------------
# 定义函数 rnn_backward，用于计算整个序列的 RNN 前向传播梯度
def rnn_backward(da, caches):
    """
    实现 RNN 对整个序列的反向传播

    参数说明:
    da -- 上游梯度，损失函数对所有隐藏状态 a 的梯度，形状 (n_a, m, T_x)
    caches -- rnn_forward 前向传播时保存的缓存，包含每个时间步的 cache 和原始输入 x

    返回值:
    gradients -- 字典，包含：
                        dx -- 对输入 x 的梯度，形状 (n_x, m, T_x)
                        da0 -- 对初始隐藏状态 a0 的梯度，形状 (n_a, m)
                        dWax -- 对输入到隐藏权重的梯度，形状 (n_a, n_x)
                        dWaa -- 对隐藏到隐藏权重的梯度，形状 (n_a, n_a)
                        dba -- 对偏置 b 的梯度，形状 (n_a, 1)
    """

    # ------------------1. 从 caches 中获取缓存------------------
    # caches = (list of caches, x)
    caches, x = caches
    # 获取第一个时间步的缓存，用于获取维度信息
    a1, a0, x1, parameters = caches[0]

    # ------------------2. 获取维度信息------------------
    n_a, m, T_x = da.shape  # n_a: 隐藏状态维度, m: 样本数, T_x: 时间步数
    n_x, m = x1.shape       # n_x: 输入特征数, m: 样本数

    # ------------------3. 初始化梯度------------------
    dx = np.zeros([n_x, m, T_x])   # 输入梯度
    dWax = np.zeros([n_a, n_x])    # 输入到隐藏权重梯度
    dWaa = np.zeros([n_a, n_a])    # 隐藏到隐藏权重梯度
    dba = np.zeros([n_a, 1])       # 偏置梯度
    da0 = np.zeros([n_a, m])       # 初始隐藏状态梯度
    da_prevt = np.zeros([n_a, m])  # 前一时间步隐藏状态梯度，用于链式累加

    # ------------------4. 反向遍历所有时间步------------------
    # 从最后一个时间步 T_x-1 开始，依次回到第 0 个时间步
    for t in reversed(range(T_x)):
        # 计算时间步 t 的梯度
        # da[:, :, t]: 当前时间步上游梯度
        # da_prevt: 下一时间步传回的梯度
        gradients = rnn_cell_backward(da[:, :, t] + da_prevt, caches[t])

        # ------------------5. 从 rnn_cell_backward 返回的字典中获取各梯度------------------
        dxt = gradients["dxt"]          # 当前输入 xt 的梯度
        da_prevt = gradients["da_prev"] # 前一隐藏状态 a_prev 的梯度
        dWaxt = gradients["dWax"]       # 当前时间步输入权重梯度
        dWaat = gradients["dWaa"]       # 当前时间步隐藏权重梯度
        dbat = gradients["dba"]         # 当前时间步偏置梯度

        # ------------------6. 累加梯度------------------
        # 保存 dx
        dx[:, :, t] = dxt
        # 对权重和偏置梯度进行时间步累加
        dWax += dWaxt
        dWaa += dWaat
        dba += dbat

    # ------------------7. 设置初始隐藏状态梯度------------------
    # 反向传播结束后，da_prevt 就是 da0 的梯度
    da0 = da_prevt

    # ------------------8. 将所有梯度封装到字典------------------
    gradients = {
        "dx": dx,
        "da0": da0,
        "dWax": dWax,
        "dWaa": dWaa,
        "dba": dba
    }

    # 返回整个序列的梯度
    return gradients


In [94]:
# ------------------1. 固定随机种子------------------
# 保证随机数可复现，方便调试和教学
np.random.seed(1)

# ------------------2. 生成随机输入序列------------------
# x: 输入序列，形状 (n_x, m, T_x) = (3, 10, 4)
# n_x = 3 输入特征数
# m = 10 样本数量（batch size）
# T_x = 4 时间步长度
x = np.random.randn(3,10,4)

# a0: 初始隐藏状态，形状 (n_a, m) = (5, 10)
a0 = np.random.randn(5,10)

# ------------------3. 初始化 RNN 权重与偏置------------------
# 输入到隐藏权重
Wax = np.random.randn(5,3)
# 隐藏到隐藏权重
Waa = np.random.randn(5,5)
# 隐藏到输出权重
Wya = np.random.randn(2,5)
# 隐藏层偏置
ba = np.random.randn(5,1)
# 输出偏置
by = np.random.randn(2,1)

# 封装参数字典
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}

# ------------------4. 前向传播------------------
# a: 所有时间步隐藏状态，形状 (n_a, m, T_x) = (5, 10, 4)
# y: 所有时间步输出预测值，形状 (n_y, m, T_x) = (2, 10, 4)
# caches: 前向传播缓存
a, y, caches = rnn_forward(x, a0, parameters)

# ------------------5. 生成随机上游梯度------------------
# da: 损失对隐藏状态 a 的梯度，形状 (n_a, m, T_x) = (5, 10, 4)
da = np.random.randn(5, 10, 4)

# ------------------6. 调用 rnn_backward 计算梯度------------------
# gradients: 字典，包含 dx、da0、dWax、dWaa、dba
gradients = rnn_backward(da, caches)

# ------------------7. 打印梯度值和形状进行验证------------------
# dx: 输入 x 的梯度，形状 (n_x, m, T_x) = (3, 10, 4)
print("gradients[\"dx\"][1][2] =", gradients["dx"][1][2])  # 第2个特征，第3个样本
print("gradients[\"dx\"].shape =", gradients["dx"].shape)

# da0: 初始隐藏状态 a0 的梯度，形状 (n_a, m) = (5, 10)
print("gradients[\"da0\"][2][3] =", gradients["da0"][2][3])  # 第3个隐藏单元，第4个样本
print("gradients[\"da0\"].shape =", gradients["da0"].shape)

# dWax: 输入到隐藏权重的梯度，形状 (n_a, n_x) = (5, 3)
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])  # 第4个隐藏单元，第2个输入特征
print("gradients[\"dWax\"].shape =", gradients["dWax"].shape)

# dWaa: 隐藏到隐藏权重的梯度，形状 (n_a, n_a) = (5, 5)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])  # 第2行第3列元素
print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape)

# dba: 偏置梯度，形状 (n_a, 1) = (5, 1)
print("gradients[\"dba\"][4] =", gradients["dba"][4])  # 第5个隐藏单元的偏置梯度
print("gradients[\"dba\"].shape =", gradients["dba"].shape)


gradients["dx"][1][2] = [-2.07101689 -0.59255627  0.02466855  0.01483317]
gradients["dx"].shape = (3, 10, 4)
gradients["da0"][2][3] = -0.3149423751266506
gradients["da0"].shape = (5, 10)
gradients["dWax"][3][1] = 11.264104496527771
gradients["dWax"].shape = (5, 3)
gradients["dWaa"][1][2] = 2.303333126579893
gradients["dWaa"].shape = (5, 5)
gradients["dba"][4] = [-0.74747722]
gradients["dba"].shape = (5, 1)


**Expected Output**:

<table>
    <tr>
        <td>
            **gradients["dx"][1][2]** =
        </td>
        <td>
           [-2.07101689 -0.59255627  0.02466855  0.01483317]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dx"].shape** =
        </td>
        <td>
           (3, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"][2][3]** =
        </td>
        <td>
           -0.314942375127
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
         <tr>
        <td>
            **gradients["dWax"][3][1]** =
        </td>
        <td>
           11.2641044965
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWax"].shape** =
        </td>
        <td>
           (5, 3)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"][1][2]** = 
        </td>
        <td>
           2.30333312658
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"].shape** =
        </td>
        <td>
           (5, 5)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"][4]** = 
        </td>
        <td>
           [-0.74747722]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

## 3.2 - LSTM 的反向传播（Backward Pass）


### 3.2.1 单步反向传播（One Step Backward）

LSTM 的反向传播比前向传播略复杂。  
下面提供了 LSTM 单步反向传播的所有公式（如果你喜欢微积分练习，也可以尝试自己推导）。

---

### 3.2.2 门的梯度（Gate Derivatives）

$$
d \Gamma_o^{\langle t \rangle} = da_{next} * \tanh(c_{next}) * \Gamma_o^{\langle t \rangle} * (1-\Gamma_o^{\langle t \rangle}) \tag{7}
$$

$$
d \tilde c^{\langle t \rangle} = dc_{next} * \Gamma_u^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * i_t * da_{next} * \tilde c^{\langle t \rangle} * (1-\tanh(\tilde c)^2) \tag{8}
$$

$$
d\Gamma_u^{\langle t \rangle} = dc_{next} * \tilde c^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * \tilde c^{\langle t \rangle} * da_{next} * \Gamma_u^{\langle t \rangle} * (1-\Gamma_u^{\langle t \rangle}) \tag{9}
$$

$$
d\Gamma_f^{\langle t \rangle} = dc_{next} * \tilde c_{prev} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * c_{prev} * da_{next} * \Gamma_f^{\langle t \rangle} * (1-\Gamma_f^{\langle t \rangle}) \tag{10}
$$

---

### 3.2.3 参数梯度（Parameter Derivatives）

$$
dW_f = d\Gamma_f^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t \end{pmatrix}^T \tag{11}
$$
$$
dW_u = d\Gamma_u^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t \end{pmatrix}^T \tag{12}
$$
$$
dW_c = d\tilde c^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t \end{pmatrix}^T \tag{13}
$$
$$
dW_o = d\Gamma_o^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t \end{pmatrix}^T \tag{14}
$$

- 计算偏置梯度 $db_f, db_u, db_c, db_o$ 时，只需沿水平方向（axis=1）对 $d\Gamma_f^{\langle t \rangle}, d\Gamma_u^{\langle t \rangle}, d\tilde c^{\langle t \rangle}, d\Gamma_o^{\langle t \rangle}$ 求和，并设置 `keepdims=True`。

---

- 计算前一隐藏状态、前一记忆状态和输入的梯度：

$$
da_{prev} = W_f^T * d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle} + W_c^T * d\tilde c^{\langle t \rangle} + W_o^T * d\Gamma_o^{\langle t \rangle} \tag{15}
$$
> 注意公式 15 中的权重取前 $n_a$ 行（即 $W_f[:n_a,:]$ 等）。

$$
dc_{prev} = dc_{next} * \Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1-\tanh(c_{next})^2) * \Gamma_f^{\langle t \rangle} * da_{next} \tag{16}
$$

$$
dx^{\langle t \rangle} = W_f^T * d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle} + W_c^T * d\tilde c^{\langle t \rangle} + W_o^T * d\Gamma_o^{\langle t \rangle} \tag{17}
$$
> 公式 17 中的权重取从 $n_a$ 到最后的行（即 $W_f[n_a:,:]$ 等）。

---

**练习**：根据公式 $7-17$ 实现 `lstm_cell_backward` 函数。  
祝你好运！ :)


In [95]:
# ------------------LSTM 单步反向传播函数------------------
# 定义函数 lstm_cell_backward，用于计算 LSTM 单个时间步的梯度
def lstm_cell_backward(da_next, dc_next, cache):
    """
    实现 LSTM 单元在单个时间步的反向传播

    参数说明:
    da_next -- 损失函数对下一隐藏状态 a_next 的梯度，形状 (n_a, m)
    dc_next -- 损失函数对下一记忆状态 c_next 的梯度，形状 (n_a, m)
    cache -- 来自 lstm_cell_forward 的缓存，包含 (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    返回值:
    gradients -- 字典，包含各参数的梯度：
                        dxt -- 当前时间步输入的梯度，形状 (n_x, m)
                        da_prev -- 前一隐藏状态 a_prev 的梯度，形状 (n_a, m)
                        dc_prev -- 前一记忆状态 c_prev 的梯度，形状 (n_a, m)
                        dWf/dWi/dWc/dWo -- 各门权重梯度，形状 (n_a, n_a + n_x)
                        dbf/dbi/dbc/dbo -- 各门偏置梯度，形状 (n_a, 1)
    """

    # ------------------1. 从 cache 中获取前向传播保存的值------------------
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache

    # 获取输入和隐藏状态的维度
    n_x, m = xt.shape      # n_x: 输入特征数, m: 样本数
    n_a, m = a_next.shape  # n_a: 隐藏状态维度

    # ------------------2. 计算各门的梯度------------------
    # dot: 输出门的梯度，公式 7
    dot = da_next * np.tanh(c_next) * ot * (1 - ot)

    # dcct: 候选记忆单元的梯度，公式 8
    dcct = (dc_next * it + ot * (1 - np.square(np.tanh(c_next))) * it * da_next) * (1 - np.square(cct))

    # dit: 更新门的梯度，公式 9
    dit = (dc_next * cct + ot * (1 - np.square(np.tanh(c_next))) * cct * da_next) * it * (1 - it)

    # dft: 遗忘门的梯度，公式 10
    dft = (dc_next * c_prev + ot * (1 - np.square(np.tanh(c_next))) * c_prev * da_next) * ft * (1 - ft)

    # ------------------3. 计算各门权重和偏置梯度------------------
    # 将 a_prev 和 xt 拼接，用于矩阵运算
    concat = np.concatenate((a_prev, xt), axis=0).T  # 形状 (m, n_a + n_x)
    
    # 权重梯度（公式 11-14）
    dWf = np.dot(dft, concat)
    dWi = np.dot(dit, concat)
    dWc = np.dot(dcct, concat)
    dWo = np.dot(dot, concat)
    
    # 偏置梯度
    dbf = np.sum(dft, axis=1, keepdims=True)
    dbi = np.sum(dit, axis=1, keepdims=True)
    dbc = np.sum(dcct, axis=1, keepdims=True)
    dbo = np.sum(dot, axis=1, keepdims=True)

    # ------------------4. 计算 da_prev, dc_prev, dxt ------------------
    # da_prev: 对前一隐藏状态的梯度，公式 15
    da_prev = (
        np.dot(parameters["Wf"][:, :n_a].T, dft) +
        np.dot(parameters["Wc"][:, :n_a].T, dcct) +
        np.dot(parameters["Wi"][:, :n_a].T, dit) +
        np.dot(parameters["Wo"][:, :n_a].T, dot)
    )

    # dc_prev: 对前一记忆状态的梯度，公式 16
    dc_prev = dc_next * ft + ot * (1 - np.square(np.tanh(c_next))) * ft * da_next

    # dxt: 对输入 xt 的梯度，公式 17
    dxt = (
        np.dot(parameters["Wf"][:, n_a:].T, dft) +
        np.dot(parameters["Wc"][:, n_a:].T, dcct) +
        np.dot(parameters["Wi"][:, n_a:].T, dit) +
        np.dot(parameters["Wo"][:, n_a:].T, dot)
    )

    # ------------------5. 将梯度封装到字典------------------
    gradients = {
        "dxt": dxt,
        "da_prev": da_prev,
        "dc_prev": dc_prev,
        "dWf": dWf, "dbf": dbf,
        "dWi": dWi, "dbi": dbi,
        "dWc": dWc, "dbc": dbc,
        "dWo": dWo, "dbo": dbo
    }

    # 返回所有梯度
    return gradients


In [96]:
np.random.seed(1)  # 设置随机种子，保证每次生成的随机数一致，便于复现结果

# ------------------生成随机输入数据和隐藏状态------------------
xt = np.random.randn(3,10)      # 当前时间步的输入 xt，形状 (n_x=3, m=10)
a_prev = np.random.randn(5,10)  # 前一隐藏状态 a_prev，形状 (n_a=5, m=10)
c_prev = np.random.randn(5,10)  # 前一记忆状态 c_prev，形状 (n_a=5, m=10)

# ------------------生成 LSTM 参数------------------
Wf = np.random.randn(5, 5+3)    # 遗忘门权重矩阵，形状 (n_a, n_a+n_x)
bf = np.random.randn(5,1)       # 遗忘门偏置，形状 (n_a,1)
Wi = np.random.randn(5, 5+3)    # 更新门权重矩阵
bi = np.random.randn(5,1)       # 更新门偏置
Wo = np.random.randn(5, 5+3)    # 输出门权重矩阵
bo = np.random.randn(5,1)       # 输出门偏置
Wc = np.random.randn(5, 5+3)    # 候选记忆单元权重矩阵
bc = np.random.randn(5,1)       # 候选记忆单元偏置
Wy = np.random.randn(2,5)       # 输出权重矩阵
by = np.random.randn(2,1)       # 输出偏置

# 将参数封装成字典
parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, 
              "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

# ------------------前向传播，获取缓存------------------
a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)
# a_next: 下一隐藏状态 (5,10)
# c_next: 下一记忆状态 (5,10)
# yt: 输出预测 (2,10)
# cache: 用于反向传播保存的中间变量

# ------------------生成下一步梯度，测试反向传播------------------
da_next = np.random.randn(5,10)  # 下一隐藏状态梯度
dc_next = np.random.randn(5,10)  # 下一记忆状态梯度

# 调用 LSTM 单步反向传播函数
gradients = lstm_cell_backward(da_next, dc_next, cache)

# ------------------打印各梯度值及形状，便于观察------------------
print("gradients[\"dxt\"][1][2] =", gradients["dxt"][1][2])  # 输入梯度 dxt
print("gradients[\"dxt\"].shape =", gradients["dxt"].shape)  # 输入梯度形状

print("gradients[\"da_prev\"][2][3] =", gradients["da_prev"][2][3])  # 前一隐藏状态梯度
print("gradients[\"da_prev\"].shape =", gradients["da_prev"].shape)

print("gradients[\"dc_prev\"][2][3] =", gradients["dc_prev"][2][3])  # 前一记忆状态梯度
print("gradients[\"dc_prev\"].shape =", gradients["dc_prev"].shape)

print("gradients[\"dWf\"][3][1] =", gradients["dWf"][3][1])  # 遗忘门权重梯度
print("gradients[\"dWf\"].shape =", gradients["dWf"].shape)

print("gradients[\"dWi\"][1][2] =", gradients["dWi"][1][2])  # 更新门权重梯度
print("gradients[\"dWi\"].shape =", gradients["dWi"].shape)

print("gradients[\"dWc\"][3][1] =", gradients["dWc"][3][1])  # 候选记忆权重梯度
print("gradients[\"dWc\"].shape =", gradients["dWc"].shape)

print("gradients[\"dWo\"][1][2] =", gradients["dWo"][1][2])  # 输出门权重梯度
print("gradients[\"dWo\"].shape =", gradients["dWo"].shape)

print("gradients[\"dbf\"][4] =", gradients["dbf"][4])  # 遗忘门偏置梯度
print("gradients[\"dbf\"].shape =", gradients["dbf"].shape)

print("gradients[\"dbi\"][4] =", gradients["dbi"][4])  # 更新门偏置梯度
print("gradients[\"dbi\"].shape =", gradients["dbi"].shape)

print("gradients[\"dbc\"][4] =", gradients["dbc"][4])  # 候选记忆偏置梯度
print("gradients[\"dbc\"].shape =", gradients["dbc"].shape)

print("gradients[\"dbo\"][4] =", gradients["dbo"][4])  # 输出门偏置梯度
print("gradients[\"dbo\"].shape =", gradients["dbo"].shape)


gradients["dxt"][1][2] = 3.2305591151091884
gradients["dxt"].shape = (3, 10)
gradients["da_prev"][2][3] = -0.06396214197109241
gradients["da_prev"].shape = (5, 10)
gradients["dc_prev"][2][3] = 0.7975220387970015
gradients["dc_prev"].shape = (5, 10)
gradients["dWf"][3][1] = -0.1479548381644968
gradients["dWf"].shape = (5, 8)
gradients["dWi"][1][2] = 1.0574980552259903
gradients["dWi"].shape = (5, 8)
gradients["dWc"][3][1] = 2.3045621636876668
gradients["dWc"].shape = (5, 8)
gradients["dWo"][1][2] = 0.33131159528921084
gradients["dWo"].shape = (5, 8)
gradients["dbf"][4] = [0.18864637]
gradients["dbf"].shape = (5, 1)
gradients["dbi"][4] = [-0.40142491]
gradients["dbi"].shape = (5, 1)
gradients["dbc"][4] = [0.25587763]
gradients["dbc"].shape = (5, 1)
gradients["dbo"][4] = [0.13893342]
gradients["dbo"].shape = (5, 1)


**Expected Output**:

<table>
    <tr>
        <td>
            **gradients["dxt"][1][2]** =
        </td>
        <td>
           3.23055911511
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dxt"].shape** =
        </td>
        <td>
           (3, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"][2][3]** =
        </td>
        <td>
           -0.0639621419711
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
         <tr>
        <td>
            **gradients["dc_prev"][2][3]** =
        </td>
        <td>
           0.797522038797
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dc_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"][3][1]** = 
        </td>
        <td>
           -0.147954838164
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"].shape** =
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"][1][2]** = 
        </td>
        <td>
           1.05749805523
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWc"][3][1]** = 
        </td>
        <td>
           2.30456216369
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWc"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWo"][1][2]** = 
        </td>
        <td>
           0.331311595289
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWo"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbf"][4]** = 
        </td>
        <td>
           [ 0.18864637]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbf"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbi"][4]** = 
        </td>
        <td>
           [-0.40142491]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbi"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"][4]** = 
        </td>
        <td>
           [ 0.25587763]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"][4]** = 
        </td>
        <td>
           [ 0.13893342]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### 3.3 LSTM 的反向传播（Backward Pass）

这一部分与之前实现的 `rnn_backward` 函数非常相似。  
你需要先创建与返回变量同维度的梯度变量，然后从序列末尾开始，**倒序遍历每个时间步**：

1. 在每个时间步调用你实现的 `lstm_cell_backward` 单步反向传播函数；
2. 将计算得到的新梯度累加到已有梯度中，逐步更新参数；
3. 保存输入梯度 `dx`，但不要更新原来的输入。

---

**实现步骤**：

实现 `lstm_backward` 函数：
- 使用 `for` 循环从 $T_x$ 倒序遍历每个时间步；
- 调用 `lstm_cell_backward`，并累加更新各梯度变量；
- 返回一个包含所有新梯度的字典。


In [97]:
def lstm_backward(da, caches):
    
    """
    实现LSTM网络的反向传播
    
    参数：
        da -- 关于隐藏状态的梯度，维度为(n_a, m, T_x)
        cachses -- 前向传播保存的信息
    
    返回：
        gradients -- 包含了梯度信息的字典：
                        dx -- 输入数据的梯度，维度为(n_x, m，T_x)
                        da0 -- 先前的隐藏状态的梯度，维度为(n_a, m)
                        dWf -- 遗忘门的权值的梯度，维度为(n_a, n_a + n_x)
                        dbf -- 遗忘门的偏置的梯度，维度为(n_a, 1)
                        dWi -- 更新门的权值的梯度，维度为(n_a, n_a + n_x)
                        dbi -- 更新门的偏置的梯度，维度为(n_a, 1)
                        dWc -- 第一个“tanh”的权值的梯度，维度为(n_a, n_a + n_x)
                        dbc -- 第一个“tanh”的偏置的梯度，维度为(n_a, n_a + n_x)
                        dWo -- 输出门的权值的梯度，维度为(n_a, n_a + n_x)
                        dbo -- 输出门的偏置的梯度，维度为(n_a, 1)
        
    """

    # 从caches中获取第一个cache（t=1）的值
    caches, x = caches
    (a1, c1, a0, c0, f1, i1, cc1, o1, x1, parameters) = caches[0]
    
    # 获取da与x1的维度信息
    n_a, m, T_x = da.shape
    n_x, m = x1.shape
    
    # 初始化梯度
    dx = np.zeros([n_x, m, T_x])
    da0 = np.zeros([n_a, m])
    da_prevt = np.zeros([n_a, m])
    dc_prevt = np.zeros([n_a, m])
    dWf = np.zeros([n_a, n_a + n_x])
    dWi = np.zeros([n_a, n_a + n_x])
    dWc = np.zeros([n_a, n_a + n_x])
    dWo = np.zeros([n_a, n_a + n_x])
    dbf = np.zeros([n_a, 1])
    dbi = np.zeros([n_a, 1])
    dbc = np.zeros([n_a, 1])
    dbo = np.zeros([n_a, 1])
    
    # 处理所有时间步
    for t in reversed(range(T_x)):
        # 使用lstm_cell_backward函数计算所有梯度
        gradients = lstm_cell_backward(da[:,:,t],dc_prevt,caches[t])
        # 保存相关参数
        dx[:,:,t] = gradients['dxt']
        dWf = dWf+gradients['dWf']
        dWi = dWi+gradients['dWi']
        dWc = dWc+gradients['dWc']
        dWo = dWo+gradients['dWo']
        dbf = dbf+gradients['dbf']
        dbi = dbi+gradients['dbi']
        dbc = dbc+gradients['dbc']
        dbo = dbo+gradients['dbo']
    # 将第一个激活的梯度设置为反向传播的梯度da_prev。
    da0 = gradients['da_prev']

    # 保存所有梯度到字典变量内
    gradients = {"dx": dx, "da0": da0, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}
    
    return gradients

In [98]:
# 设置随机种子，保证结果可复现
np.random.seed(1)

# 生成模拟输入数据 x，形状为 (n_x=3, m=10, T_x=7)
# 3: 每个时间步输入特征维度，10: batch size, 7: 时间步数
x = np.random.randn(3,10,7)

# 初始化隐藏状态 a0，形状为 (n_a=5, m=10)
a0 = np.random.randn(5,10)

# 初始化 LSTM 权重和偏置
# 遗忘门权重 Wf，形状 (n_a=5, n_a+n_x=5+3)
Wf = np.random.randn(5, 5+3)
# 遗忘门偏置 bf，形状 (n_a=5, 1)
bf = np.random.randn(5,1)

# 更新门权重 Wi，形状 (n_a=5, n_a+n_x=5+3)
Wi = np.random.randn(5, 5+3)
# 更新门偏置 bi，形状 (n_a=5, 1)
bi = np.random.randn(5,1)

# 输出门权重 Wo，形状 (n_a=5, n_a+n_x=5+3)
Wo = np.random.randn(5, 5+3)
# 输出门偏置 bo，形状 (n_a=5, 1)
bo = np.random.randn(5,1)

# 候选记忆门权重 Wc，形状 (n_a=5, n_a+n_x=5+3)
Wc = np.random.randn(5, 5+3)
# 候选记忆门偏置 bc，形状 (n_a=5, 1)
bc = np.random.randn(5,1)

# 将权重和偏置封装到字典中，便于传入 LSTM 函数
parameters = {
    "Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc,
    "Wy": Wy,  # 输出层权重（上一节定义的 Wy）
    "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by  # 输出层偏置
}

# 调用 lstm_forward 执行前向传播
# 输出 a: 所有时间步的隐藏状态, y: 所有时间步的预测, c: 所有时间步的记忆状态, caches: 保存中间值
a, y, c, caches = lstm_forward(x, a0, parameters)

# 生成模拟隐藏状态梯度 da，形状 (n_a=5, m=10, T_x=4)
da = np.random.randn(5, 10, 4)

# 调用 lstm_backward 执行反向传播
# 返回值 gradients 是一个字典，包含输入梯度、参数梯度等
gradients = lstm_backward(da, caches)

# 输出 dx 的部分元素和形状
print("gradients[\"dx\"][1][2] =", gradients["dx"][1][2])  # 输入第2行、第3列的梯度
print("gradients[\"dx\"].shape =", gradients["dx"].shape)   # dx 的整体形状 (n_x, m, T_x)

# 输出初始隐藏状态 da0 的部分元素和形状
print("gradients[\"da0\"][2][3] =", gradients["da0"][2][3])  # 隐状态梯度第3行、第4列
print("gradients[\"da0\"].shape =", gradients["da0"].shape)  # da0 的形状 (n_a, m)

# 输出遗忘门权重梯度 dWf 的部分元素和形状
print("gradients[\"dWf\"][3][1] =", gradients["dWf"][3][1])
print("gradients[\"dWf\"].shape =", gradients["dWf"].shape)

# 输出更新门权重梯度 dWi 的部分元素和形状
print("gradients[\"dWi\"][1][2] =", gradients["dWi"][1][2])
print("gradients[\"dWi\"].shape =", gradients["dWi"].shape)

# 输出候选记忆门权重梯度 dWc 的部分元素和形状
print("gradients[\"dWc\"][3][1] =", gradients["dWc"][3][1])
print("gradients[\"dWc\"].shape =", gradients["dWc"].shape)

# 输出输出门权重梯度 dWo 的部分元素和形状
print("gradients[\"dWo\"][1][2] =", gradients["dWo"][1][2])
print("gradients[\"dWo\"].shape =", gradients["dWo"].shape)

# 输出遗忘门偏置梯度 dbf 的部分元素和形状
print("gradients[\"dbf\"][4] =", gradients["dbf"][4])
print("gradients[\"dbf\"].shape =", gradients["dbf"].shape)

# 输出更新门偏置梯度 dbi 的部分元素和形状
print("gradients[\"dbi\"][4] =", gradients["dbi"][4])
print("gradients[\"dbi\"].shape =", gradients["dbi"].shape)

# 输出候选记忆门偏置梯度 dbc 的部分元素和形状
print("gradients[\"dbc\"][4] =", gradients["dbc"][4])
print("gradients[\"dbc\"].shape =", gradients["dbc"].shape)

# 输出输出门偏置梯度 dbo 的部分元素和形状
print("gradients[\"dbo\"][4] =", gradients["dbo"][4])
print("gradients[\"dbo\"].shape =", gradients["dbo"].shape)


gradients["dx"][1][2] = [-0.00173313  0.08287442 -0.30545663 -0.43281115]
gradients["dx"].shape = (3, 10, 4)
gradients["da0"][2][3] = -0.09591150195400465
gradients["da0"].shape = (5, 10)
gradients["dWf"][3][1] = -0.06981985612744011
gradients["dWf"].shape = (5, 8)
gradients["dWi"][1][2] = 0.10237182024854771
gradients["dWi"].shape = (5, 8)
gradients["dWc"][3][1] = -0.062498379492745254
gradients["dWc"].shape = (5, 8)
gradients["dWo"][1][2] = 0.04843891314443012
gradients["dWo"].shape = (5, 8)
gradients["dbf"][4] = [-0.0565788]
gradients["dbf"].shape = (5, 1)
gradients["dbi"][4] = [-0.15399065]
gradients["dbi"].shape = (5, 1)
gradients["dbc"][4] = [-0.29691142]
gradients["dbc"].shape = (5, 1)
gradients["dbo"][4] = [-0.29798344]
gradients["dbo"].shape = (5, 1)


**Expected Output**:

<table>
    <tr>
        <td>
            **gradients["dx"][1][2]** =
        </td>
        <td>
           [-0.00173313  0.08287442 -0.30545663 -0.43281115]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dx"].shape** =
        </td>
        <td>
           (3, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"][2][3]** =
        </td>
        <td>
           -0.095911501954
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"][3][1]** = 
        </td>
        <td>
           -0.0698198561274
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"].shape** =
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"][1][2]** = 
        </td>
        <td>
           0.102371820249
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWc"][3][1]** = 
        </td>
        <td>
           -0.0624983794927
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWc"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWo"][1][2]** = 
        </td>
        <td>
           0.0484389131444
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWo"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbf"][4]** = 
        </td>
        <td>
           [-0.0565788]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbf"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbi"][4]** = 
        </td>
        <td>
           [-0.06997391]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbi"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"][4]** = 
        </td>
        <td>
           [-0.27441821]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"][4]** = 
        </td>
        <td>
           [ 0.16532821]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### 恭喜！🎉

恭喜你完成本次作业！  
你现在已经理解了循环神经网络（RNN）的工作原理！

接下来，我们将进入下一练习：使用 RNN 构建**字符级语言模型（character-level language model）**。
